In [2]:
from sqlite3 import connect

In [4]:
with connect('test.db') as conn:
    cur = conn.cursor()
    cur.execute('create table points (x int, y int)')
    cur.execute('insert into points (x, y) values (1, 1)')
    cur.execute('insert into points (x, y) values (1, 2)')
    cur.execute('insert into points (x, y) values (2, 1)')
    for row in cur.execute('select x, y from points'):
        print(row)
    for row in cur.execute('select sum(x * y) from points'):
        print(row)
    cur.execute('drop table points')

(1, 1)
(1, 2)
(2, 1)
(5,)


In [7]:
#with ctx() as x:
#    pass

# x = ctx().__enter__()
# try:
#     pass
# finally:
#     x.__exit()

class temptable:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        cur.execute('create table points (x int, y int)')
    def __exit__(self, *args):
        cur.execute('drop table points')


with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (1, 2)')
        cur.execute('insert into points (x, y) values (2, 1)')
        for row in cur.execute('select x, y from points'):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)
    
    
    

(1, 1)
(1, 2)
(2, 1)
(5,)


In [19]:
#  can the exit evry be called before enter?
#   No!   it is all about sequencing to enforce that which is what generators are


def temptable(cur):
    cur.execute('create table points (x int, y int)')
    yield
    cur.execute('drop table points')
    
    
class contextmanager:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        self.gen = temptable(self.cur)
        next(self.gen)
    def __exit__(self, *args):
        next(self.gen, None)

with connect('test.db') as conn:
    cur = conn.cursor()
    with contextmanager(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (1, 2)')
        cur.execute('insert into points (x, y) values (2, 1)')
        for row in cur.execute('select x, y from points'):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)
    
 

(1, 1)
(1, 2)
(2, 1)
(5,)


In [15]:
# in case anything goes wrong this deletes the table
with connect('test.db') as conn:
    cur = conn.cursor()
    cur.execute('drop table points')

In [21]:
# now lets make it more generic
def temptable(cur):
    cur.execute('create table points (x int, y int)')
    yield
    cur.execute('drop table points')
    
# much more generic
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

with connect('test.db') as conn:
    cur = conn.cursor()
    with contextmanager(temptable)(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (1, 2)')
        cur.execute('insert into points (x, y) values (2, 1)')
        for row in cur.execute('select x, y from points'):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)
    
 

(1, 1)
(1, 2)
(2, 1)
(5,)


In [22]:
# now lets make it more generic
# much more generic
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

# now wrap it
def temptable(cur):
    cur.execute('create table points (x int, y int)')
    yield
    cur.execute('drop table points')
temptable = contextmanager(temptable)    

        
with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (1, 2)')
        cur.execute('insert into points (x, y) values (2, 1)')
        for row in cur.execute('select x, y from points'):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)
    
 

(1, 1)
(1, 2)
(2, 1)
(5,)


In [23]:
# now lets make it more generic
# much more generic
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

@contextmanager
def temptable(cur):
    cur.execute('create table points (x int, y int)')
    yield
    cur.execute('drop table points')

        
with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (1, 2)')
        cur.execute('insert into points (x, y) values (2, 1)')
        for row in cur.execute('select x, y from points'):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)
    
 

(1, 1)
(1, 2)
(2, 1)
(5,)


In [25]:
# use the library
from contextlib import contextmanager

@contextmanager
def temptable(cur):
    cur.execute('create table points (x int, y int)')
    try:
        yield
    finally:
        cur.execute('drop table points')

        
with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (1, 2)')
        cur.execute('insert into points (x, y) values (2, 1)')
        for row in cur.execute('select x, y from points'):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)
    
 

(1, 1)
(1, 2)
(2, 1)
(5,)
